This transform has limitations:
1. It assumes that the Initial and Ending Rates are in units of B/M or M/M
2. It only works on individual phase forecasts, not ratios

Last modified: March 21, 2024

In [ ]:
import pandas as pd
import numpy as np
from config.config_loader import get_config
import AnalyticsAndDBScripts.sql_connect as sql
import AnalyticsAndDBScripts.sql_schemas as schema

In [ ]:
# Load configs
sql_creds_dict = get_config('credentials', 'sql1_sa')

# Create distinct dictionaries for each database
sql_aries_creds_dict = sql_creds_dict.copy()
sql_aries_creds_dict['db_name'] = 'Analytics_Aries'
sql_creds_dict['db_name'] = 'Analytics'

In [ ]:
qualifier_name = 'CONDUIT'

statement = f'''
SELECT      E.*, M.WELL_ID AS WellID
FROM        dbo.AC_ECONOMIC E
LEFT JOIN   dbo.AC_PROPERTY M
ON          E.PROPNUM = M.PROPNUM
WHERE       E.QUALIFIER = '{qualifier_name}'
AND         E.SECTION = 4
ORDER BY    E.SEQUENCE
'''

In [ ]:
# Execute query and store results in a dataframe
engine = sql.sql_connect(
    username=sql_aries_creds_dict['username'], 
    password=sql_aries_creds_dict['password'], 
    db_name=sql_aries_creds_dict['db_name'], 
    server_name=sql_aries_creds_dict['servername'], 
    port=sql_aries_creds_dict['port']
)
try:
    fcst_df = pd.read_sql(statement, engine)
finally:
    engine.dispose()

In [ ]:
# Extract values from fcst_df EXPRESSION strings associated with forecast start dates
start_df = fcst_df.copy()
start_df['Measure'] = start_df.groupby('PROPNUM')['KEYWORD'].shift(-1)
start_df['Measure'] = start_df['Measure'].replace('WTR', 'WATER')
start_df = start_df[start_df['KEYWORD'] == 'START']
start_df['StartDate'] = pd.to_datetime(start_df['EXPRESSION'], format='%m/%Y')
start_df['StartDate'] = start_df['StartDate'] + pd.offsets.MonthEnd(1)
start_df = start_df.rename(columns={'QUALIFIER': 'Analyst'})
start_df = start_df[['PROPNUM', 'WellID', 'Analyst', 'Measure', 'StartDate']]

In [ ]:
# Extract values from fcst_df EXPRESSION strings associated with arps forecasts
measure_df = fcst_df[fcst_df['KEYWORD'].isin(['OIL', 'GAS', 'WTR', '"'])].copy()
measure_df['KEYWORD'] = measure_df.groupby('PROPNUM')['KEYWORD'].shift(1).where(measure_df['KEYWORD'] == '"', measure_df['KEYWORD'])
measure_df = measure_df.sort_values(['PROPNUM', 'KEYWORD', 'SEQUENCE'], ascending=[True, True, False])
measure_df['fcst_index'] = measure_df.groupby(['PROPNUM', 'KEYWORD'])['KEYWORD'].cumcount() + 1
measure_df = measure_df.sort_values(['PROPNUM', 'KEYWORD', 'SEQUENCE'])
measure_df['EXPRESSION'] = measure_df['EXPRESSION'].str.split(' ')
measure_df[['InitialRate', 'EndingRate', 'RateUnits', 'LimitValue', 'LimitUnits', 'ForecastMethod', 'ForecastMethodValue']] = pd.DataFrame(measure_df['EXPRESSION'].tolist(), index=measure_df.index)
measure_df[['InitialRate', 'EndingRate', 'RateUnits', 'LimitValue']] = measure_df[['InitialRate', 'EndingRate', 'RateUnits', 'LimitValue']].replace('X', np.nan)
measure_df['b_factor'] = measure_df['ForecastMethod'].str.extract(r'B/(\d+\.?\d*)')
measure_df['ForecastMethod'] = measure_df['ForecastMethod'].str.replace(r'B/\d+(\.\d+)?', 'HYP', regex=True)

# Convert columns to numeric
cols_to_convert = ['InitialRate', 'EndingRate', 'LimitValue', 'ForecastMethodValue', 'b_factor']
measure_df[cols_to_convert] = measure_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Extract values for Q1, Q2, Q3, Qabn, Dei, Def, t1, and t2
measure_df['Q1'] = np.where(
    (measure_df['ForecastMethod'] == 'EXP') & (measure_df['fcst_index'] > 2),
    measure_df['InitialRate'], np.nan
)
measure_df['Q2'] = np.where(
    (measure_df['ForecastMethod'] == 'EXP') & (measure_df['fcst_index'] > 2),
    measure_df['EndingRate'], np.nan
)
measure_df['Q3'] = np.where((measure_df['fcst_index'] <= 2), measure_df['InitialRate'], np.nan)
measure_df['Qabn'] = np.where((measure_df['fcst_index'] == 1), measure_df['EndingRate'], 1.0)
measure_df['Dei'] = np.where((measure_df['fcst_index'] <= 2), measure_df['ForecastMethodValue'], np.nan)
measure_df['Def'] = np.where((measure_df['fcst_index'] == 1), measure_df['ForecastMethodValue'], np.nan)
measure_df['t1'] = np.where(
    (measure_df['ForecastMethod'] == 'EXP') & (measure_df['fcst_index'] > 2) & (measure_df['LimitUnits'] == 'MO'), 
    measure_df['LimitValue'], np.nan
)
measure_df['t2'] = np.where(
    (measure_df['ForecastMethod'] == 'EXP') & (measure_df['fcst_index'] > 2) & (measure_df['LimitUnits'] == 'IMO'), 
    measure_df['LimitValue'], np.nan
)
measure_df['Q1'] = np.where(measure_df['RateUnits'].isin(['B/M', 'M/M']), measure_df['Q1'] / 30.42, measure_df['Q1'])
measure_df['Q2'] = np.where(measure_df['RateUnits'].isin(['B/M', 'M/M']), measure_df['Q2'] / 30.42, measure_df['Q2'])
measure_df['Q3'] = np.where(measure_df['RateUnits'].isin(['B/M', 'M/M']), measure_df['Q3'] / 30.42, measure_df['Q3'])
measure_df['Qabn'] = np.where(measure_df['RateUnits'].isin(['B/M', 'M/M']), measure_df['Qabn'] / 30.42, measure_df['Qabn'])

# Aggregate to fit the proper schema
drop_columns = [
    'SECTION', 'SEQUENCE', 'EXPRESSION', 'fcst_index', 'InitialRate', 'EndingRate', 
    'RateUnits', 'LimitValue', 'LimitUnits', 'ForecastMethod', 'ForecastMethodValue'
]
forecast_df = measure_df.drop(columns=drop_columns)
forecast_df = forecast_df.groupby(['PROPNUM', 'WellID', 'QUALIFIER', 'KEYWORD']).agg('max').reset_index()

# Replace KEYWORD value == WTR with WATER
forecast_df['KEYWORD'] = forecast_df['KEYWORD'].replace('WTR', 'WATER')

# Rename QUALIFIER column to Analyst and KEYWORD to Measure
forecast_df = forecast_df.rename(columns={'QUALIFIER': 'Analyst', 'KEYWORD': 'Measure'})

# Set minimum value of Qabn to 1.0.
forecast_df['Qabn'] = forecast_df['Qabn'].clip(lower=1.0)

# Divide Dei and Def columns by 100.0
forecast_df[['Dei', 'Def']] = forecast_df[['Dei', 'Def']].div(100.0)

# Add a units column. When Measure == GAS, units = MCF. When Measure == OIL, units = BBL.
forecast_df['Units'] = np.where(forecast_df['Measure'] == 'GAS', 'MCF', 'BBL')

# Join StartDate column from start_df to forecast_df
forecast_df = pd.merge(forecast_df, start_df, how='left', on=['PROPNUM', 'WellID', 'Analyst', 'Measure'])
forecast_df['DateCreated'] = pd.to_datetime('today')

In [ ]:
# Select and reorder columns
cols = ['WellID', 'Measure', 'Units', 'StartDate', 'Q1', 'Q2', 'Q3', 'Qabn', 'Dei', 'b_factor', 'Def', 't1', 't2', 'Analyst', 'DateCreated']
forecast_df = forecast_df[cols]

# Convert NaN to None for proper database insertion
forecast_df = forecast_df.where(pd.notnull(forecast_df), None)

In [ ]:
# Load well_df into dbo.FORECAST_STAGE table in SQL Server
sql.load_data_to_sql(forecast_df, sql_creds_dict, schema.forecast_stage)

# Move data from dbo.FORECAST_STAGE to dbo.FORECAST and drop dbo.FORECAST_STAGE
sql.execute_stored_procedure(sql_creds_dict, 'sp_InsertFromStagingToForecast')